In [1]:
import tensorflow as tf
import os
import os.path
import shutil
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
LOGDIR = "mnist_demo/"
LABELS = os.path.join(os.getcwd(), "labels_1024.tsv")
SPRITES = os.path.join(os.getcwd(), "sprite_1024.png")

In [4]:
SPRITES

'/media/luis/media/Libros,Ebooks,Manuales,Tutoriales,Lecturas/udacity/deep-learning-nanodegree/projects/tensorboard/sprite_1024.png'

In [5]:
def make_hparam_string(learning_rate, conv1_depth,conv2_depth,fc1_size,fc2_size):

  return "lr_"+str(learning_rate)+",conv1depth_"+str(conv1_depth)+",conv2depth_"+str(conv2_depth)+",fc1size_"+str(fc1_size)+",fc2size_"+str(fc2_size)

In [6]:
make_hparam_string(1,2,3,4,5)

'lr_1,conv1depth_2,conv2depth_3,fc1size_4,fc2size_5'

In [7]:
def conv_layer(input,channels_in,channels_out,name ="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5,5,channels_in,channels_out] ,stddev=0.1),name ="W")
        b = tf.Variable(tf.constant(0.1,shape= [channels_out]),name ="B")
        conv = tf.nn.conv2d(input,w,strides = [1,1,1,1],padding="SAME")
        act = tf.nn.relu(conv+b)
        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("activations",act)

        return act

In [8]:
def fc_layer(input,channels_in,channels_out,name ="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([channels_in,channels_out],stddev=0.1),name ="W")
        b = tf.Variable(tf.constant(0.1,shape=[channels_out]),name="B")
        act = tf.nn.relu(tf.matmul(input,w) + b)

        return act

In [9]:
def train_mnist(learning_rate, conv1_depth,conv2_depth,fc1_size,fc2_size):
    tf.reset_default_graph()
    param_string  = make_hparam_string(learning_rate,conv1_depth,conv2_depth,fc1_size,fc2_size)
    print("Starting training for:"+param_string)
    
    x = tf.placeholder(tf.float32,shape=[None,784],name="x")
    y = tf.placeholder(tf.float32,shape=[None,10],name="labels")
    x_image = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',x_image,3)

    conv1 = conv_layer(x_image,1,conv1_depth,"conv1")
    pool1 = tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

    conv2 = conv_layer(pool1,conv1_depth,conv2_depth,"conv2")
    pool2 = tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    flattened = tf.reshape(pool2,[-1,7*7*conv2_depth])

    fc1 = fc_layer(flattened,7*7*conv2_depth,fc1_size,"fc1")
    logits = fc_layer(fc1,fc1_size,fc2_size,"fc2")

    with tf.name_scope("cross_entropy"):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
        tf.summary.scalar('cross_entropy',cross_entropy)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar("accuracy",accuracy)

    with tf.Session() as sess:
        merged_summary =  tf.summary.merge_all()
        #the size of this tensor will be:n,d where n = size of data points to be displayed, d = embedding dimension, in this example n = size of test images passed
        embedding = tf.Variable(tf.zeros([1024,fc1_size]),name ="test_embedding") 
        assigment = embedding.assign(fc1)
        
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        
        writer = tf.summary.FileWriter(LOGDIR+param_string)
        writer.add_graph(sess.graph)
        

        config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
        embedding_config = config.embeddings.add()
        embedding_config.tensor_name = embedding.name
        embedding_config.sprite.image_path = SPRITES
        embedding_config.metadata_path = LABELS
        
        embedding_config.sprite.single_image_dim.extend([28,28])
        tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer,config)
        
        for i in range(2001):
            batch = mnist.train.next_batch(100)

            if i % 50 == 0:
                [train_accuracy] = sess.run([accuracy],feed_dict =  {x:batch[0] , y:batch[1]})
                summary = sess.run(merged_summary , feed_dict={x:batch[0],y:batch[1]})
                sess.run(assigment,feed_dict={x:mnist.test.images[:1024],y:mnist.test.labels[:1024]})
                if i%500 == 0:
                    print("step %d ,training accuracy %g" % (i,train_accuracy))
                writer.add_summary(summary,i)
                saver.save(sess,os.path.join(LOGDIR+param_string,"model.ckpt"),i)

            sess.run(train_step,feed_dict = {x:batch[0],y:batch[1]})

In [10]:
learning_rates = [1E-3,1E-4,1E-5]
conv1_depths = [16,32,64]
conv2_depths = [32,64,128]
fc1_sizes = [512,1024,2048]



In [11]:
#train_mnist(learning_rates[0],32,64,1024,10)

In [12]:
for learning_rate in learning_rates:
    for conv1_depth in conv1_depths:
        for conv2_depth in conv2_depths:
            for fc1_size in fc1_sizes:
                train_mnist(learning_rate,conv1_depth,conv2_depth,fc1_size,10)

Starting training for:lr_0.001,conv1depth_16,conv2depth_32,fc1size_512,fc2size_10
step 0 ,training accuracy 0.07
step 500 ,training accuracy 0.42
step 1000 ,training accuracy 0.3
step 1500 ,training accuracy 0.42
step 2000 ,training accuracy 0.43
Starting training for:lr_0.001,conv1depth_16,conv2depth_32,fc1size_1024,fc2size_10
step 0 ,training accuracy 0.13
step 500 ,training accuracy 0.12
step 1000 ,training accuracy 0.13
step 1500 ,training accuracy 0.07
step 2000 ,training accuracy 0.12
Starting training for:lr_0.001,conv1depth_16,conv2depth_32,fc1size_2048,fc2size_10
step 0 ,training accuracy 0.1
step 500 ,training accuracy 0.09
step 1000 ,training accuracy 0.12
step 1500 ,training accuracy 0.14
step 2000 ,training accuracy 0.13
Starting training for:lr_0.001,conv1depth_16,conv2depth_64,fc1size_512,fc2size_10
step 0 ,training accuracy 0.12
step 500 ,training accuracy 0.39
step 1000 ,training accuracy 0.33
step 1500 ,training accuracy 0.38
step 2000 ,training accuracy 0.38
Starting

Starting training for:lr_0.0001,conv1depth_16,conv2depth_128,fc1size_512,fc2size_10
step 0 ,training accuracy 0.1
step 500 ,training accuracy 0.05
step 1000 ,training accuracy 0.13


KeyboardInterrupt: 

Launch tensorboard passing --logdir <path used in FileWriter> :
tensorboard --logdir /tmp/mnist_demo/2

* To use tensorboard, you create a tf.summary.FileWriter("logs path") which is used later to write to a log your graph,scalars,histograms,images, audio or other information.
* To write the network's graph to the logs, use(assuming tensorflow session is called sess): 
        writer = tf.summary.FileWriter("log dir")
        writer.add_graph(sess.graph)
* The default generated graph its very complex and provides low level details, to get a cleaner and simplear graph, we need to group operations together, this is done using name scopes, a group of related operations is assigned a name scope using tf.name_scope,  for example for a fully conected layer, we would group operations like:
        with tf.name_scope(name):
            w = tf.Variable(tf.truncated_normal([channels_in,channels_out],stddev=0.1),name ="W")
            b = tf.Variable(tf.constant(0.1,shape=[channels_out]),name="B")
            act = tf.nn.relu(tf.matmul(input,w) + b)
* Tensorboard can show other data and how it evolves in training(for example cost in time) using distinct types of summaries, you just create new summaries,for every summary pass a name, and a tensorflow node that you want to display. The types of summaries are:
    * Scalars: for scalar numbers like cost,accuracy,or other performance metrics(precision,recall,right predictions,wrong predictions, etc) we use Scalars , for example if you have a tensorflow node called cost  you will use:
            cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
            tf.summary.scalar('cross_entropy',cross_entropy)
      This will create a new chart in the "Scalars" tab of tensorboard
    * Histograms: for multivalued variables, for example a weights matrix or vector, to see its distribution, we create a histogram summary , this creates a chart in the histograms tab, as well as the distributions tab,for example to get the histogram and distribution of a biases vector:
             b = tf.Variable(tf.constant(0.1,shape= [channels_out]),name ="B")
             tf.summary.histogram("biases",b)